<a href="https://colab.research.google.com/github/sgpt94/Games-Recommender/blob/main/games_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Gameplay_Data.csv to Gameplay_Data.csv


In [ ]:
import io
game = pd.read_csv(io.BytesIO(uploaded['Gameplay_Data.csv']))

In [ ]:
game.head()

,Uid,Eventtime,Entry_Fee,Win_Loss,Winnings,Tournament_Type,Num_Players
0,90976392B3AA5DB94D134936B765879E,1607294430194,0.0,W,10.0,D,4
1,7457E894E1F7F1938E55802B964BDD27,1607286707669,1000.0,W,1700.0,A,2
2,52E43F75D7393581C4E2C5EDC7D24671,1607023262845,3500.0,L,0.0,A,2
3,5736C06C66C6BC7036CCE9804F54A57C,1607778875873,1000.0,W,3200.0,B,4
4,5FE2DDA792CA72F731D818A5ECC5F1EB,1607778574193,2500.0,W,4200.0,A,2


In [ ]:
game['type'] = game.apply(lambda x: str(x['Tournament_Type'])+'_'+str(int(x['Entry_Fee'])),axis=1)

In [ ]:
data =  game.groupby(['Uid','type'],as_index=False)['Win_Loss'].size()

In [ ]:
data.rename(columns = {'Uid':'user','size':'plays','type':'game'}, inplace = True)

In [ ]:
data.head()

,user,game,plays
0,0012CB6DA8CB360BA4ACC6A40F2C5E7D,A_100,30
1,0012CB6DA8CB360BA4ACC6A40F2C5E7D,A_1000,3
2,0012CB6DA8CB360BA4ACC6A40F2C5E7D,D_0,16
3,0016F36E9D87DC1C1D8FCA7F84648122,A_100,58
4,0016F36E9D87DC1C1D8FCA7F84648122,A_1000,5


In [ ]:
data.shape

(31635, 3)

In [ ]:
#median of games played in a each game_type
data['plays'].quantile([0.5,0.6,0.7,0.8,0.9,0.95])

0.50     3.0
0.60     4.0
0.70     7.0
0.80    11.0
0.90    23.0
0.95    40.0
Name: plays, dtype: float64

In [ ]:
#median of games played by each user
data.groupby('user')['plays'].sum().quantile([0.5,0.6,0.7,0.8,0.9,0.95])

0.50      6.0
0.60     13.0
0.70     23.0
0.80     40.0
0.90     78.0
0.95    127.0
Name: plays, dtype: float64

In [ ]:
data['user_id'] = data['user'].astype("category").cat.codes
data['game_id'] = data['game'].astype("category").cat.codes
 
 # Create a lookup frame so we can get the game names back in 
 # readable form later.
item_lookup = data[['game_id', 'game']].drop_duplicates()
item_lookup['game_id'] = item_lookup.game_id.astype(str)
 
data = data.drop(['user', 'game'], axis=1)

In [ ]:
data.head()

,plays,user_id,game_id
0,30,0,1
1,3,0,2
2,16,0,18
3,58,1,1
4,5,1,2


In [ ]:
!pip install pyspark

In [ ]:
from pyspark.mllib.recommendation import ALS

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [ ]:
def get_mat_sparsity(df):
    # Count the total number of ratings in the dfset
    count_nonzero = df.select("plays").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = df.select("user").distinct().count() * df.select("game").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The df dfframe is ", "%.2f" % sparsity + "% sparse.")
    

The df dfframe is  85.65% sparse.


In [ ]:
df = spark.createDataFrame(data)

In [ ]:
# df1 = spark.createDataFrame(data)

In [ ]:
# Create test and train set
(train, test) = df.randomSplit([0.8, 0.2], seed = 2020)

In [ ]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ALS model
als = ALS(
         userCol="user_id", 
         itemCol="game_id",
         ratingCol="plays", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop")

In [ ]:
# Import the requisite packages
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [ ]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="plays", 
           predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [ ]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)
#Extract best model from the cv model above
best_model = model.bestModel
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

24.08207005813972


In [ ]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 10
  MaxIter: 10
  RegParam: 0.1


In [ ]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(20)
nrecommendations.limit(20).show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{1, 56.36848}, {...|
|      3|[{16, 1.0121588},...|
|      5|[{16, 1.0121588},...|
|      6|[{16, 1.0121588},...|
|      9|[{2, 0.9949498}, ...|
|     12|[{1, 270.9105}, {...|
|     13|[{4, 4.1492777}, ...|
|     15|[{2, 1.980379}, {...|
|     16|[{1, 11.800024}, ...|
|     17|[{16, 1.0121588},...|
|     19|[{1, 15.526955}, ...|
|     20|[{16, 2.0243175},...|
|     22|[{16, 1.0121588},...|
|     27|[{4, 34.498165}, ...|
|     28|[{19, 5.664467}, ...|
|     31|[{1, 15.5544615},...|
|     34|[{4, 18.601896}, ...|
|     35|[{1, 28.8329}, {1...|
|     37|[{16, 3.1107795},...|
|     40|[{16, 4.4172}, {1...|
+-------+--------------------+



In [ ]:
from pyspark.sql.functions import explode, col
nrecommendations = nrecommendations.withColumn("rec_exp", explode("recommendations")).select('user_id', col("rec_exp.game_id"), col("rec_exp.rating"))

nrecommendations.limit(20).show()

+-------+-------+----------+
|user_id|game_id|    rating|
+-------+-------+----------+
|      1|      1|  56.36848|
|      1|     16|  37.77243|
|      1|     18| 14.549601|
|      1|     14|  4.973921|
|      1|      2| 4.9612308|
|      1|     17| 2.2995095|
|      1|     20| 1.9976994|
|      1|      6|  1.930826|
|      1|      4| 1.5215834|
|      1|     13| 1.0235052|
|      1|     12| 0.9808467|
|      1|      0| 0.5891949|
|      1|     19|0.57583904|
|      1|     15|0.49112818|
|      1|     10|0.42752075|
|      1|      9|0.41980264|
|      1|     11|0.38489622|
|      1|      8|0.29088098|
|      1|      5|0.21524091|
|      1|      7|0.11296943|
+-------+-------+----------+



In [ ]:
reco = nrecommendations.toPandas()

In [ ]:
data.head()

,user_id,game_id,plays
0,0,1,30
1,0,2,3
2,0,18,16
3,1,1,58
4,1,2,5


In [ ]:
reco.head()

,user_id,game_id,rating,game
0,1,1,56.368481,A_100
1,1,16,37.772430,C_3500
2,1,18,14.549601,D_0
3,1,14,4.973921,C_10000
4,1,2,4.961231,A_1000


In [ ]:
reco = reco.merge(item_lookup,how='left',on='game_id')

In [ ]:
data[data['user_id']==0]

,plays,user_id,game_id
0,30,0,1
1,3,0,2
2,16,0,18


In [ ]:
odf = reco.merge(data,how='left',left_on=['user_id','game_id'],right_on=['user_id','game_id'])

In [ ]:
odf = odf[['user_id','game_id','game','plays','rating']]
odf.rename(columns = {'rating':'pred_plays', 'old_col2':'new_col2'}, inplace = True)

In [ ]:
odf.head()

,user_id,game_id,game,plays,pred_plays
0,1,1,A_100,58.0,56.368481
1,1,16,C_3500,NaN,37.772430
2,1,18,D_0,NaN,14.549601
3,1,14,C_10000,NaN,4.973921
4,1,2,A_1000,5.0,4.961231


In [ ]:
odf[(odf['user_id']==5) & (odf['plays'].isnull())].sort_values(by=['pred_plays'], ascending=False).head(5)

,user_id,game_id,game,plays,pred_plays
40,5,16,C_3500,NaN,1.012159
42,5,10,A_50000,NaN,0.707990
43,5,3,A_10000,NaN,0.600076
44,5,4,A_100000,NaN,0.374229
45,5,1,A_100,NaN,0.264040


In [ ]:
item_lookup.dtypes

game_id     int64
game       object
dtype: object

In [ ]:
item_lookup['game_id'] = pd.to_numeric(item_lookup['game_id'])

In [ ]:
tdf = test.toPandas()

In [ ]:
tdf.head()

,plays,user_id,game_id
0,1,12,6
1,1,16,13
2,1,18,8
3,1,23,8
4,1,23,9


In [ ]:
tdf = tdf.merge(reco,how='left',left_on=['user_id','game_id'],right_on=['user_id','game_id'])